# Day 16: Permutation Promenade

You come upon a very unusual sight; a group of programs here appear to be [dancing](https://www.youtube.com/watch?v=lyZQPjUT5B4&t=53).

There are sixteen programs in total, named `a` through `p`. They start by standing in a line: `a` stands in position `0`, `b` stands in position `1`, and so on until `p`, which stands in position `15`.

The programs' *dance* consists of a sequence of *dance moves*:

* *Spin*, written `sX`, makes `X` programs move from the end to the front, but maintain their order otherwise. (For example, `s3` on `abcde` produces `cdeab`).
* *Exchange*, written `xA/B`, makes the programs at positions `A` and `B` swap places.
* *Partner*, written `pA/B`, makes the programs named `A` and `B` swap places.

For example, with only five programs standing in a line (`abcde`), they could do the following dance:

* `s1`, a spin of size `1`: `eabcd`.
* `x3/4`, swapping the last two programs: `eabdc`.
* `pe/b`, swapping programs `e` and `b`: `baedc`.

After finishing their dance, the programs end up in order `baedc`.

You watch the dance for a while and record their dance moves (your puzzle input). In what order are the programs standing after their dance?

## Solution to part 1

First, lets define the programs for the challenge, and the test case, as well as get the dance moves for the challenge and the test case:

In [1]:
programs = list("abcdefghijklmnop")
challenge = open('input/day16.txt').read().strip().split(',')

test_programs = list("abcde")
test_case = ['s1', 'x3/4', 'pe/b']

Then I created a simple state machine to represent the dance:

In [2]:
class Dance:
    def __init__(self, programs, moves):
        self.programs = programs
        self.moves = moves

    def spin(self, offset):
        self.programs = self.programs[-offset:] + self.programs[:-offset]

    def exchange(self, pos_a, pos_b):
        self.programs[pos_a], self.programs[pos_b] = self.programs[pos_b], self.programs[pos_a]

    def partner(self, a, b):
        pos_a, pos_b = self.programs.index(a), self.programs.index(b)
        self.exchange(pos_a, pos_b)

    def dance(self):
        for move in self.moves:
            if move.startswith('s'):
                self.spin(int(move[1:]))
            elif move.startswith('x'):
                a, b = list(int(x) for x in move[1:].split('/'))
                self.exchange(a, b)
            elif move.startswith('p'):
                a, b = move[1:].split('/')
                self.partner(a, b)

I then ran the test,

In [3]:
test_dance = Dance(test_programs, test_case)
test_dance.dance()
assert test_dance.programs == list('baedc')

and then I computed the solution for part 1:

In [4]:
dance = Dance(programs[:], challenge)
dance.dance()
print(''.join(dance.programs))

iabmedjhclofgknp


# Part Two

Now that you're starting to get a feel for the dance moves, you turn your attention to *the dance as a whole*.

Keeping the positions they ended up in from their previous dance, the programs perform it again and again: including the first dance, a total of *one billion* (`1000000000`) times.

In the example above, their second dance would *begin* with the order `baedc`, and use the same dance moves:

* `s1`, a spin of size `1`: `cbaed`.
* `x3/4`, swapping the last two programs: `cbade`.
* `pe/b`, swapping programs `e` and `b`: `ceadb`.

*In what order are the programs standing* after their billion dances?

## Solution to part 2

Unfortunately, while the previous solution works well for part 1, it takes 50ms to complete. Multiplying this with one billion suggests a solution time of about 579 days. :(

It seems the solution is to look for cycles in the dance routine.

I had to sneak a look at some solutions in order to get this.

In [5]:
def part2(programs, moves, repeat=1000000000):
    dance = Dance(programs, moves)
    memo = []

    for i in range(repeat):
        state = ''.join(dance.programs)

        if state in memo:
            return memo[repeat % i]

        memo.append(state)
        dance.dance()

In [6]:
part2(programs[:], challenge)

'oildcmfeajhbpngk'